Start by defining a `Python` function that we want to compute.

In [1]:
def f(a, b, c):
    return (a & b) ^ c

Generate a circuit that computes this function. To implement the logical operations we use  standard verilog gates, which are available in `mantle.verilog.gates`.

In [2]:
import magma as m
import mantle

class VerilatorExample(m.Circuit):
    IO = ["a", m.In(m.Bit), "b", m.In(m.Bit), "c", m.In(m.Bit), "d", m.Out(m.Bit)]
    
    @classmethod
    def definition(io):
        m.wire(f(io.a, io.b, io.c), io.d)

m.compile("build/VerilatorExample", VerilatorExample, "coreir-verilog")
%cat build/VerilatorExample.v



module corebit_and (
  input in0,
  input in1,
  output out
);
  assign out = in0 & in1;

endmodule //corebit_and

module corebit_xor (
  input in0,
  input in1,
  output out
);
  assign out = in0 ^ in1;

endmodule //corebit_xor

module VerilatorExample (
  input  a,
  input  b,
  input  c,
  output  d
);
  //Wire declarations for instance 'inst0' (Module corebit_and)
  wire  inst0__in0;
  wire  inst0__in1;
  wire  inst0__out;
  corebit_and inst0(
    .in0(inst0__in0),
    .in1(inst0__in1),
    .out(inst0__out)
  );

  //Wire declarations for instance 'inst1' (Module corebit_xor)
  wire  inst1__in0;
  wire  inst1__in1;
  wire  inst1__out;
  corebit_xor inst1(
    .in0(inst1__in0),
    .in1(inst1__in1),
    .out(inst1__out)
  );

  //All the connections
  assign inst0__in0 = a;
  assign inst0__in1 = b;
  assign inst1__in0 = inst0__out;
  assign inst1__in1 = c;
  assign d = inst1__out;

endmodule //VerilatorExample


Next, generate a verilator test harness in `C++` for the circuit. The test vectors are generated using the python function `f`. The verilator test bench compares the output of the simulator to those test vectors.

In [3]:
from fault.test_vectors import generate_function_test_vectors
from fault.verilator_target import VerilatorTarget

test_vectors = generate_function_test_vectors(VerilatorExample, f)

VerilatorTarget(VerilatorExample, test_vectors).run()
%cat build/test_VerilatorExample.cpp

#include "VVerilatorExample.h"
#include "verilated.h"
#include <cassert>
#include <iostream>

typedef struct {
    unsigned int value;
    bool is_any_value;
} value_t;

void check(const char* port, int a, value_t b, int i) {
    if (!b.is_any_value) {
        std::cerr << port << "=" << b.value << ", ";
    }
    if (!b.is_any_value && !(a == b.value)) {
        std::cerr << std::endl;  // end the current line
        std::cerr << "Got      : " << a << std::endl;
        std::cerr << "Expected : " << b.value << std::endl;
        std::cerr << "i        : " << i << std::endl;
        std::cerr << "Port     : " << port << std::endl;

        exit(1);
    }
}

int main(int argc, char **argv, char **env) {
    Verilated::commandArgs(argc, argv);
    VVerilatorExample* top = new VVerilatorExample;

    value_t tests[9][4] = {
        {  // 0
            {0b0, false},  // a
            {0b0, false},  // b
            {0b0, false},  // c
            {0, true},  // d
        },
        {  // 